## Data Scraping - entradas.com

Como primera fuente he decididio escoger la web de 'entradas.com', seleccionando específicamente conciertos y festivales que estos tienen disponibles. 

En cuanto al proceso de scrapeo que se lleva a cabo, he decicido centrarme en los siguientes datos: 
- Artista.
- Número de eventos que tienen disponibles.
- Fechas, en este caso algunos eventos presentan dos fechas o fechas en ese rango, por lo que muestro ambas fechas ('fecha1' y 'fecha2')
- Precio de salida de las entradas al evento. 
- Enlace al evento. 

Para comenzar con la búsqueda, decido seguir el método selenium de scrapeo. 

Para ello, importo las librerías que necesito, y aquellas funciones que pueden ser útiles en mi búsqueda. 

In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver # driver de selenium
import time   # para manejar tiempo

In [2]:
from selenium.webdriver.chrome.options import Options   # opciones del driver
from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...
from selenium.webdriver.support.ui import WebDriverWait   # es para esperar
from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...
from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click
from selenium.webdriver.common.keys import Keys  # manejar teclas

In [6]:
# Además, hago la 'driver configuration'

opciones = Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

#### 1. Conecto con la página WEB

In [4]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                                # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

In [5]:
# Tenemos 5 páginas en total, por lo que a partir de aquí iremos llamando a la url para que cambie de página. 

url = 'https://www.entradas.com/city/madrid-370/conciertos-y-festivales-85/'

driver.get(url)

- Conecto con la Web de entradas.com para trabajar sobre esta, y hacer scraping de los datos que quiero. Utilizaré el método de Scrapeo de selenium.

#### 2. Aceptar las cookies 

- Lo primero que hago es darle a Aceptas las cookies del sitio web. 
- Para ello tengo un problema inicial, ya que está categorizada como una pestaña "shadow" por lo que cuando busco la opción de hacer click al botón de aceptar, aparece como una pestaña / lista vacía. 

In [7]:
div_shadow = driver.find_element(By.XPATH, '//div[@class="cmpwrapper"]')

In [8]:
# Check if the element has a Shadow DOM using JavaScript
has_shadow_dom = driver.execute_script('return arguments[0].shadowRoot !== null', div_shadow)

if has_shadow_dom:
    print("Element has a Shadow DOM")
else:
    print("Element does not have a Shadow DOM")

Element has a Shadow DOM


In [9]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', div_shadow)

In [10]:
shadow_root.find_elements(By.CSS_SELECTOR, '#cmpwelcomebtnyes > a')[0].click()

- Una vez consigo traspasar la barrera del 'Shadow DOM', entonces puedo comenzar con el scrapeo de las tablas que necesito para mi estudio. 

#### 3. Empezamos el scrapeo por eventos en tabla

In [11]:
tabla = '//product-group-item'

In [12]:
table = driver.find_elements(By.XPATH, tabla)
table

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_26")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_27")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_28")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_29")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_30")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79

In [13]:
type(table)

list

In [14]:
elementos = driver.find_elements(By.XPATH, '//*[@id="list-item-0"]')
elementos

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_46")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_47")>]

In [15]:
type(elementos)

list

In [16]:
elementos = elementos[0].text.split('\n')
elementos

['Cariño',
 'Inverfest 2024',
 '13/01/20241 Evento',
 ' desde 19,80 €',
 'Mostrar 1 Evento']

In [17]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_48")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_49")>]

In [18]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//*[@class = "btn theme-button-radius btn-sm btn-block btn-primary theme-interaction-btn-bg theme-btn-font-color theme-btn-font-color-hover"]')
    enlaces = [enl.get_attribute('href') for enl in enlace]

    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Cariño', 'Iván Ferreiro', 'The World Of Hans Zimmer', 'Triángulo de Amor Bizarro', 'Nico B', 'Cala Vento', 'Sidonie', 'Belako - Sigo regando', 'Biznaga', 'Morreo - Alegría Club!', 'Diego el Cigala - Obras Maestras', 'Symphonic Rhapsody of Queen', 'Sexy Zebras', 'Biografía - Rafa Sánchez, de la unión', 'OBK', 'El Consorcio - Toda una vida', 'Rock en Familia - Hijos del Rock and Roll', 'Rock en Familia - I Love Rock and Roll', 'Rock en Familia - Queenmanía', 'Rock en Familia - I Love Heavy Metal'], ['13/01/2024', '1 Evento', '13/01/2024', '1 Evento', '24/04/2024 ─ 29/04/2024', '2 eventos', '29/12/2023', '1 Evento', '14/01/2024', '1 Evento', '01/12/2023', '1 Evento', '12/01/2024 ─ 14/01/2024', '2 eventos', '20/01/2024', '1 Evento', '26/01/2024', '1 Evento', '18/01/2024', '1 Evento', '11/12/2023 ─ 18/12/2023', '2 eventos', '16/01/2024', '1 Evento', '01/02/2024', '1 Evento', '11/12/2023', '1 Evento', '13/01/2024 ─ 27/01/2024', '3 eventos', '27/11/2023 ─ 28/11/2023', '2 eventos', '05/11/2

In [19]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [20]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024', '13/01/2024', '24/04/2024 ─ 29/04/2024', '29/12/2023', '14/01/2024', '01/12/2023', '12/01/2024 ─ 14/01/2024', '20/01/2024', '26/01/2024', '18/01/2024', '11/12/2023 ─ 18/12/2023', '16/01/2024', '01/02/2024', '11/12/2023', '13/01/2024 ─ 27/01/2024', '27/11/2023 ─ 28/11/2023', '05/11/2023 ─ 10/03/2024', '03/12/2023', '17/12/2023 ─ 14/04/2024', '19/11/2023 ─ 24/03/2024']
['1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '3 eventos', '2 eventos', '2 eventos', '1 Evento', '2 eventos', '2 eventos']


In [21]:
import pandas as pd

In [22]:
datos_1 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
datos_1

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €",https://www.entradas.com/artist/sidonie/sidoni...
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/belako/
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €",https://www.entradas.com/artist/biznaga/
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €",https://www.entradas.com/artist/morreo/


#### Para entrar en la siguiente página (Pág. 2)

In [23]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[6]/a/i')[0].click()

In [24]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [25]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_48")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_224")>]

In [26]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Morgan', 'Los Zigarros', 'Queralt Lahoz', 'Noa', 'Mocedades - Avanzando hacia el futuro', 'John Grant', 'Callas en concierto - En holograma', 'Zenet', 'Laura Pausini', 'Concierto Sinfónico de Abono A13 (Jueves)', 'Concierto Sinfónico de Abono A7 (Viernes)', 'Concierto Sinfónico de Abono A15 (Viernes)', 'Concierto Sinfónico de Abono A19 (Viernes)', 'Concierto Sinfónico de Abono B16 (Viernes)', 'Concierto Sinfónico de Abono B18 (Jueves)', 'Concierto Sinfónico de Abono B18 (Viernes)', 'Concierto Sinfónico de Abono B12 (Viernes)', 'Concierto Sinfónico de Abono B16 (Jueves)', 'Concierto Sinfónico de Abono A15 (Jueves)', 'Concierto Sinfónico de Abono B14 (Jueves)'], ['19/01/2024', '1 Evento', '11/01/2024 ─ 12/01/2024', '2 eventos', '01/12/2023 ─ 02/12/2023', '2 eventos', '19/03/2024', '1 Evento', '08/01/2024', '1 Evento', '17/11/2023', '1 Evento', '11/11/2023 ─ 09/12/2023', '2 eventos', '26/11/2023', '1 Evento', '27/01/2024', '1 Evento', '22/02/2024', '1 Evento', '24/11/2023', '1 Evento',

In [27]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [28]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['19/01/2024', '11/01/2024 ─ 12/01/2024', '01/12/2023 ─ 02/12/2023', '19/03/2024', '08/01/2024', '17/11/2023', '11/11/2023 ─ 09/12/2023', '26/11/2023', '27/01/2024', '22/02/2024', '24/11/2023', '15/03/2024', '26/04/2024', '22/03/2024', '18/04/2024', '19/04/2024', '16/02/2024', '21/03/2024', '14/03/2024', '29/02/2024']
['1 Evento', '2 eventos', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento']


In [29]:
datos_2 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

datos_2

,artista,fechas,eventos,precios,enlaces
0,Morgan,19/01/2024,1 Evento,"desde 29,70 €",https://www.entradas.com/artist/morgan/
1,Los Zigarros,11/01/2024 ─ 12/01/2024,2 eventos,"desde 30,80 €",https://www.entradas.com/artist/los-zigarros/
2,Queralt Lahoz,01/12/2023 ─ 02/12/2023,2 eventos,"desde 23,00 €",https://www.entradas.com/artist/queralt-lahoz/
3,Noa,19/03/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/noa/
4,Mocedades - Avanzando hacia el futuro,08/01/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/mocedades/moce...
5,John Grant,17/11/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/john-grant/
6,Callas en concierto - En holograma,11/11/2023 ─ 09/12/2023,2 eventos,"desde 18,00 €",https://www.entradas.com/artist/maria-callas/
7,Zenet,26/11/2023,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/zenet/
8,Laura Pausini,27/01/2024,1 Evento,"desde 30,00 €",https://www.entradas.com/artist/laura-pausini/
9,Concierto Sinfónico de Abono A13 (Jueves),22/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


#### Para entrar en la siguiente página (Pág. 3)

In [30]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[7]/a/i')[0].click()

In [31]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [32]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_48")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_366")>]

In [33]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Concierto Sinfónico de Abono A9 (Jueves)', 'Concierto Sinfónico de Abono A17 (Viernes)', 'Concierto Sinfónico de Abono A13 (Viernes)', 'Concierto Sinfónico de Abono B8 (Jueves)', 'Concierto Sinfónico de Abono A11 (Viernes)', 'Concierto Sinfónico de Abono A7 (Jueves)', 'Concierto Sinfónico de Abono B8 (Viernes)', 'Concierto Sinfónico de Abono B6 (Viernes)', 'Concierto Sinfónico de Abono B10 (Viernes)', 'JÓVENES MÚSICOS III', 'JÓVENES MÚSICOS I', 'Concierto Sinfónico de Abono B20 (Viernes)', 'Concierto Sinfónico de Abono B10 (Jueves)', 'Concierto Sinfónico de Abono A9 (Viernes)', 'Concierto Sinfónico de Abono A17 (Jueves)', 'Concierto Sinfónico de Abono B14 (Viernes)', 'Concierto Sinfónico de Abono A19 (Jueves)', 'Ensayos Ciclo Sinfónico', 'JÓVENES MÚSICOS II', 'CONCIERTO DE GALARDONADOS EL CONCURSO JUVENTUDES MUSICALES'], ['18/01/2024', '1 Evento', '12/04/2024', '1 Evento', '23/02/2024', '1 Evento', '11/01/2024', '1 Evento', '09/02/2024', '1 Evento', '23/11/2023', '1 Evento', '12/01/

In [34]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [35]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['18/01/2024', '12/04/2024', '23/02/2024', '11/01/2024', '09/02/2024', '23/11/2023', '12/01/2024', '10/11/2023', '26/01/2024', '05/04/2024', '17/11/2023', '10/05/2024', '25/01/2024', '19/01/2024', '11/04/2024', '01/03/2024', '25/04/2024', '10/11/2023 ─ 09/05/2024', '02/02/2024', '17/05/2024']
['1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '1 Evento', '15 eventos', '1 Evento', '1 Evento']


In [36]:
datos_3 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

datos_3

,artista,fechas,eventos,precios,enlaces
0,Concierto Sinfónico de Abono A9 (Jueves),18/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
1,Concierto Sinfónico de Abono A17 (Viernes),12/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
2,Concierto Sinfónico de Abono A13 (Viernes),23/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
3,Concierto Sinfónico de Abono B8 (Jueves),11/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
4,Concierto Sinfónico de Abono A11 (Viernes),09/02/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
5,Concierto Sinfónico de Abono A7 (Jueves),23/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
6,Concierto Sinfónico de Abono B8 (Viernes),12/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
7,Concierto Sinfónico de Abono B6 (Viernes),10/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
8,Concierto Sinfónico de Abono B10 (Viernes),26/01/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...
9,JÓVENES MÚSICOS III,05/04/2024,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/orquesta-sinfo...


#### Para entrar en la siguiente página (Pág. 4)

In [37]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[7]/a/i')[0].click()

In [38]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [39]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_48")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_599")>]

In [40]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Concierto Sinfónico de Abono B20 (Jueves)', 'Concierto Sinfónico de Abono B12 (Jueves)', 'Concierto Sinfónico de Abono A11 (Jueves)', 'Paul Alone', 'McEnroe', 'Bestia Bebé', 'Ricardo Moya', 'Maximiliano Calvo - El Gallo', 'Grande Amore', 'Christina Rosenvinge', 'Rubén Pozo', 'Delgao', 'Austin TV', 'Ariel Rot - 25 años hablando solo', 'Israel B - EED1HS', 'Francisco', 'Antonio Lizana', 'Raúl Paz', 'Shai Maestro', 'Antonio Rey y amigos', 'Mark Guiliana', 'Billy Cobham Spectrum 50 Project', 'Calibro 35', 'Richard Bona', 'Matthew Halsall', 'The Cookers', 'Stick Men & Gilipojazz', "Ben L'Oncle Soul", 'Takuya Kuroda', 'Fatoumata Diawara', 'Scary Pockets', 'Theo Croker Quartet', 'Kassa Overall', 'Moisés P. Sánchez', 'Martirio & Chano Domínguez', 'El Kanka: Ruta Firestone 2023', 'Nando Agüeros', 'Jason Derulo', 'Zree and more', 'Tina Turner Sinfónico', '“Skybridges”', 'Elvis in my life', 'Concierto Conmemorativo 50 Aniversario', '“Dedication II”', 'Bloomsound', 'Lara Bello flamenco', 'Joni 

In [41]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [42]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['09/05/2024', '15/02/2024', '08/02/2024', '01/02/2024', '13/01/2024', '28/01/2024', '21/01/2024', '11/01/2024', '20/01/2024', '10/01/2024', '25/01/2024', '27/01/2024', '31/01/2024', '25/01/2024', '25/01/2024', '12/01/2024 ─ 23/03/2024', '30/11/2023', '07/11/2023', '12/11/2023', '05/11/2023', '20/11/2023', '17/11/2023', '24/11/2023', '10/11/2023', '19/11/2023', '09/11/2023', '05/11/2023', '15/11/2023', '07/11/2023', '21/11/2023', '09/02/2024', '09/11/2023', '25/11/2023', '18/11/2023', '14/11/2023', '23/11/2023', '28/01/2024', '06/12/2023 ─ 07/12/2023', '15/03/2024', '24/05/2024 ─ 25/05/2024', '08/03/2024', '26/01/2024', '01/12/2023', '16/02/2024', '24/05/2024', '12/04/2024', '31/05/2024', '19/04/2024', '02/02/2024', '22/03/2024', '05/04/2024', '10/11/2023', '21/12/2023 ─ 22/12/2023', '01/12/2023', '11/11/2023', '17/05/2024', '26/04/2024', '01/03/2024', '23/02/2024', '19/01/2024', '12/01/2024', '09/02/2024', '24/11/2023', '17/11/2023', '15/12/2023', '22/12/2023', '14/12/2023', '10/11/20

In [74]:
datos_4 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})
datos_4.head()


,artista,fechas,eventos,precios,enlaces
0,Jaleos Jondos,04/11/2023 ─ 10/12/2023,40 eventos,"desde 32,50 €",https://www.entradas.com/artist/jaleos-jondos/
1,Rodrigo y Gabriela - In Between Thoughts...A N...,07/11/2023 ─ 09/11/2023,2 eventos,"desde 30,00 €",https://www.entradas.com/artist/rodrigo-y-gabr...
2,El Conciertazo en Familia: Especial Navidad,30/12/2023,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/el-conciertazo...
3,Schumann y Brahms - Ensemble Praeteritum,16/11/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/ensemble-praet...
4,Alice Phoebe Lou,29/11/2023,1 Evento,"desde 24,00 €",https://www.entradas.com/artist/ciclo-jaguar/a...


#### Para entrar en la siguiente página (Pág. 5)

In [44]:
driver.find_elements(By.XPATH, '//div/product-listing/listing-pagination/nav/ul/pagination-item[7]/a/i')[0].click()

In [45]:
table = driver.find_elements(By.XPATH, '//product-group-item')

In [46]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_48")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33e045f46882e7842aaad8436e79d52a", element="D80BBFA91CB85365B105ABB1D0C613B9_element_1162")>]

In [47]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//div[2]/product-group-item/listing-item/article/div/listing-cta/a')
    enlaces = [enl.get_attribute('href') for enl in enlace]
    
    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Jaleos Jondos', 'Rodrigo y Gabriela - In Between Thoughts...A New World Tour 2023', 'El Conciertazo en Familia: Especial Navidad', 'Schumann y Brahms - Ensemble Praeteritum', 'Alice Phoebe Lou', 'Hard GZ: Tour 23/24', 'Felabration - Tributo a FeIa Kuti', 'Fred de Palma', 'Homenaje a Pablo Milanés', 'Momentos Alhambra: Rikas', 'Rolling Stones vs Beatles', 'La Serva Padrona - Aranjuez', 'Kaze - Gira 2024', 'Lo + actual', 'Mujer, inspiración y acción', 'Luceral + Cherry Pie', 'The Greatest Love Of All', 'Danza para Guitarra', 'Blues & Roots', 'Invisible Tattoo', 'Impulso presenta: Danilo Plessow (Motor City Drum Ensemble) + Javi Frias & Roberto Tempo', 'Aires de Persia', 'Footloosing', 'Apartamentos Acapulco + Cora Yako', 'Diego Guerrero', 'Rhythm Of The Dance', 'Luna Ki', 'Rawayana - ¿Quien trae las cornetas?', 'Rupatrupa', 'The Music of Big Star + The Sadies', 'Eva Català Group', 'Fran Perea "1 más 1 son 20"', 'Gran Gala Flamenco - Madrid', 'The Other Side - A Pink Floyd Live Experie

In [48]:
artistas = entry[0]
fecha_eventos = entry[1]
precios = entry[2]

In [49]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['04/11/2023 ─ 10/12/2023', '07/11/2023 ─ 09/11/2023', '30/12/2023', '16/11/2023', '29/11/2023', '22/02/2024', '04/11/2023', '26/03/2024 ─ 27/03/2024', '24/03/2024', '14/12/2023', '17/11/2023', '25/11/2023', '21/04/2024', '01/12/2023', '19/11/2023', '08/11/2023', '20/11/2023', '09/12/2023', '26/11/2023', '29/11/2023', '10/11/2023', '13/11/2023', '11/11/2023', '01/12/2023', '15/12/2023', '12/02/2024', '25/04/2024', '07/04/2024 ─ 10/04/2024', '17/11/2023', '16/11/2023', '25/11/2023', '23/02/2024', '24/12/2023 ─ 01/01/2024', '24/11/2023 ─ 17/12/2023', '17/12/2023', '11/11/2023', '20/04/2024', '08/12/2023', '16/02/2024', '18/11/2023', '29/02/2024', '26/11/2023', '18/11/2023', '25/02/2024', '18/11/2023', '18/11/2023', '12/03/2024', '04/11/2023', '07/11/2023', '03/12/2023', '25/04/2024 ─ 26/04/2024', '26/11/2023', '05/11/2023', '23/12/2023', '01/12/2023 ─ 02/12/2023', '19/11/2023', '24/11/2023', '08/11/2023', '17/12/2023', '07/11/2023', '11/12/2023', '04/11/2023']
['40 eventos', '2 eventos',

In [50]:
datos_5 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

,artista,fechas,eventos,precios,enlaces
0,Jaleos Jondos,04/11/2023 ─ 10/12/2023,40 eventos,"desde 32,50 €",https://www.entradas.com/artist/jaleos-jondos/
1,Rodrigo y Gabriela - In Between Thoughts...A N...,07/11/2023 ─ 09/11/2023,2 eventos,"desde 30,00 €",https://www.entradas.com/artist/rodrigo-y-gabr...
2,El Conciertazo en Familia: Especial Navidad,30/12/2023,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/el-conciertazo...
3,Schumann y Brahms - Ensemble Praeteritum,16/11/2023,1 Evento,"desde 12,00 €",https://www.entradas.com/artist/ensemble-praet...
4,Alice Phoebe Lou,29/11/2023,1 Evento,"desde 24,00 €",https://www.entradas.com/artist/ciclo-jaguar/a...
...,...,...,...,...,...
57,Larkin Poe,08/11/2023,1 Evento,"desde 25,00 €",https://www.entradas.com/artist/larkin-poe/
58,Gala de Zarzuela en Navidad,17/12/2023,1 Evento,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
59,Oneus,07/11/2023,1 Evento,"desde 55,00 €",https://www.entradas.com/artist/oneus/
60,15 Tangos de historia - La Porteña Tango - Madrid,11/12/2023,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/la-portena-tan...


In [70]:
eventos = pd.concat([datos_1, datos_2, datos_3, datos_4, datos_5])
eventos.head()

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/


In [55]:
eventos = eventos.reset_index(inplace=True)

In [ ]:
eventos.drop(columns="index",inplace=True)

In [63]:
eventos.head(20)

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €",https://www.entradas.com/artist/sidonie/sidoni...
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/belako/
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €",https://www.entradas.com/artist/biznaga/
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €",https://www.entradas.com/artist/morreo/


In [64]:
eventos.tail(20)

,artista,fechas,eventos,precios,enlaces
202,Momentos Alhambra: Mammal Hands,18/11/2023,1 Evento,"desde 27,00 €",https://www.entradas.com/artist/momentos-alham...
203,Paula Koops,25/02/2024,1 Evento,"desde 20,00 €",https://www.entradas.com/artist/paula-koops/
204,Fila 8: Clásicos del Cine Moderno - Jazz & the...,18/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/fila-8-clasico...
205,Discotizer Presenta - Cody Currie + Javi Frias,18/11/2023,1 Evento,"desde 15,00 €",https://www.entradas.com/artist/clubbing/disco...
206,Lina,12/03/2024,1 Evento,"desde 35,00 €",https://www.entradas.com/artist/lina/
207,El Amor y la Muerte en la Ópera,04/11/2023,1 Evento,"desde 35,00 €",https://www.entradas.com/artist/el-amor-y-la-m...
208,Momentos Alhambra: Laura Misch,07/11/2023,1 Evento,"desde 18,00 €",https://www.entradas.com/artist/momentos-alham...
209,"Homenaje a Morricone, Zimmer, Williams, Hisaishi",03/12/2023,1 Evento,"desde 45,00 €",https://www.entradas.com/artist/homenaje-a-mor...
210,Parov Stelar - Theater Tour,25/04/2024 ─ 26/04/2024,2 eventos,"desde 40,00 €",https://www.entradas.com/artist/parov-stelar/
211,Zetazen,26/11/2023,1 Evento,"desde 22,00 €",https://www.entradas.com/artist/zetazen/


In [77]:
eventos = pd.DataFrame(eventos)

# Separa las fechas en fecha1 y fecha2
eventos['fecha1'] = eventos['fechas'].str.split(' ─ ').str[0]
eventos['fecha2'] = eventos['fechas'].str.split(' ─ ').str[1]

# Elimina espacios en blanco adicionales en las fechas
eventos['fecha1'] = eventos['fecha1'].str.strip()
eventos['fecha2'] = eventos['fecha2'].str.strip()

# Rellena las fechas faltantes en fecha2 con NaN
eventos['fecha2'].fillna('', inplace=True)

# Elimina la columna original 'fechas' si es necesario
eventos.drop('fechas', axis=1, inplace=True)

In [78]:
eventos = eventos[['artista', 'eventos', 'fecha1', 'fecha2', 'precios', 'enlaces']]

,artista,eventos,fecha1,fecha2,precios,enlaces
0,Cariño,1 Evento,13/01/2024,,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,1 Evento,13/01/2024,,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,2 eventos,24/04/2024,29/04/2024,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,1 Evento,29/12/2023,,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,1 Evento,14/01/2024,,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
...,...,...,...,...,...,...
57,Larkin Poe,1 Evento,08/11/2023,,"desde 25,00 €",https://www.entradas.com/artist/larkin-poe/
58,Gala de Zarzuela en Navidad,1 Evento,17/12/2023,,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
59,Oneus,1 Evento,07/11/2023,,"desde 55,00 €",https://www.entradas.com/artist/oneus/
60,15 Tangos de historia - La Porteña Tango - Madrid,1 Evento,11/12/2023,,"desde 20,00 €",https://www.entradas.com/artist/la-portena-tan...


In [79]:
eventos.head()

,artista,eventos,fecha1,fecha2,precios,enlaces
0,Cariño,1 Evento,13/01/2024,,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,1 Evento,13/01/2024,,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,2 eventos,24/04/2024,29/04/2024,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,1 Evento,29/12/2023,,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,1 Evento,14/01/2024,,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
...,...,...,...,...,...,...
57,Larkin Poe,1 Evento,08/11/2023,,"desde 25,00 €",https://www.entradas.com/artist/larkin-poe/
58,Gala de Zarzuela en Navidad,1 Evento,17/12/2023,,"desde 39,00 €",https://www.entradas.com/artist/gala-de-zarzue...
59,Oneus,1 Evento,07/11/2023,,"desde 55,00 €",https://www.entradas.com/artist/oneus/
60,15 Tangos de historia - La Porteña Tango - Madrid,1 Evento,11/12/2023,,"desde 20,00 €",https://www.entradas.com/artist/la-portena-tan...


In [83]:

eventos['fecha1'] = pd.to_datetime(eventos['fecha1'], format='%d/%m/%Y', errors='coerce')
eventos['fecha2'] = pd.to_datetime(eventos['fecha2'], format='%d/%m/%Y', errors='coerce')


In [84]:

# Separar las fechas en columnas de día, mes y año
eventos['dia_fecha1'] = eventos['fecha1'].dt.day
eventos['mes_fecha1'] = eventos['fecha1'].dt.month
eventos['año_fecha1'] = eventos['fecha1'].dt.year


In [85]:
eventos['dia_fecha2'] = eventos['fecha2'].dt.day
eventos['mes_fecha2'] = eventos['fecha2'].dt.month
eventos['año_fecha2'] = eventos['fecha2'].dt.year

In [87]:
eventos.head()

,artista,eventos,fecha1,fecha2,precios,enlaces,dia_fecha1,mes_fecha1,año_fecha1,dia_fecha2,mes_fecha2,año_fecha2
0,Cariño,1 Evento,2024-01-13,NaT,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...,13,1,2024,NaN,NaN,NaN
1,Iván Ferreiro,1 Evento,2024-01-13,NaT,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/,13,1,2024,NaN,NaN,NaN
2,The World Of Hans Zimmer,2 eventos,2024-04-24,2024-04-29,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/,24,4,2024,29.0,4.0,2024.0
3,Triángulo de Amor Bizarro,1 Evento,2023-12-29,NaT,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...,29,12,2023,NaN,NaN,NaN
4,Nico B,1 Evento,2024-01-14,NaT,"desde 13,50 €",https://www.entradas.com/artist/nico-b/,14,1,2024,NaN,NaN,NaN


In [88]:
eventos.drop(['fecha1', 'fecha2'], axis=1, inplace=True)

In [97]:
nuevos_nombres = {
    'dia_fecha1': 'dia1',
    'mes_fecha1': 'mes1',
    'año_fecha1': 'año1',
    'dia_fecha2': 'dia2',
    'mes_fecha2': 'mes2',
    'año_fecha2': 'año2'
}

In [99]:
eventos = eventos.rename(columns=nuevos_nombres)
eventos.head()

,artista,eventos,Dia1,Mes1,Año1,Dia2,Mes2,Año2,precios,enlaces
0,Cariño,1 Evento,13,1,2024,,,,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,1 Evento,13,1,2024,,,,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,2 eventos,24,4,2024,29.0,4.0,2024.0,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,1 Evento,29,12,2023,,,,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,1 Evento,14,1,2024,,,,"desde 13,50 €",https://www.entradas.com/artist/nico-b/


In [95]:
# Cambio el orden de las columnas, para mas facil visualización.

column_order = ['artista', 'eventos', 'Dia1', 'Mes1', 'Año1', 'Dia2', 'Mes2', 'Año2', 'precios', 'enlaces']
eventos = eventos[column_order]

# Reemplazo los valores NaN con espacios en blanco

eventos = eventos.fillna(' ')

eventos.head()


,artista,eventos,Dia1,Mes1,Año1,Dia2,Mes2,Año2,precios,enlaces
0,Cariño,1 Evento,13,1,2024,,,,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,1 Evento,13,1,2024,,,,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,2 eventos,24,4,2024,29.0,4.0,2024.0,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,1 Evento,29,12,2023,,,,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,1 Evento,14,1,2024,,,,"desde 13,50 €",https://www.entradas.com/artist/nico-b/


In [96]:
type(d

NameError: name 'dia' is not defined

- Una vez que he terminado de trabajar con la web, salgo de esta:

In [82]:
driver.quit()

- Una vez he importado el dataframe como .csv lo marco para que no se me vuelva a convertir.

In [81]:
#eventos.to_csv('eventos.csv', index=False)